In [1]:
from tensorflow.keras import layers, Sequential
import tensorflow_datasets as tfds
import tensorflow.keras
import numpy as np
import re
import os
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2022-12-01 12:53:58.291073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nico_marais/.pyenv/versions/3.10.6/envs/forward_farming/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# this is used to remove stage direction if we don't want them
def remove_stage_dir(text):
    text = re.sub("[\<].*?[\>]", "", text)
    text = re.sub("\\s+", " ", text)
    return text

In [3]:
# this is used to remove the word "SPEECH" adn the number following after that in the corpus
def remove_SPEECH(text):
    text = re.sub("SPEECH \d+", "", text)
    text = re.sub("\\s+", " ", text)
    return text

In [4]:
path = '/Users/nico_marais/code/Liedjieboer/forward_farming/clean_data/clean_winemag_coord.csv'
text = open(path).read()
text_into_list = text.replace('\n', ' ').split(".")

In [5]:
text_into_list

[',country,description,region_1,lat,lon 4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics',
 " Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew",
 '",Willamette Valley,44',
 '942554,-122',
 '9337615 21,US,"A sleek mix of tart berry, stem and herb, along with a hint of oak and chocolate, this is a fair value in a widely available, drink-now Oregon Pinot',
 ' The wine oak-aged for six months, whether in neutral or re-staved is not indicated',
 '",Oregon,43',
 '8041334,-120',
 '5542012 25,US,"Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site',
 ' Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture',
 '",Sonoma Coast,38',
 '4242109,-123',
 '1011733 35,US,"As with many of the Erath 201

In [7]:
in_sentences = []
# read all files in the folder (need to be txt with UTF-8 encoding)
# chop it up in sentances (for Tokenizer)
# for filename in os.listdir(path):
#     text = ''.join(open(path+'/'+filename, encoding = 'cp1252', mode="r").readlines())
split_text = re.split(r' *[\.\?!][\'"\)\]]* *', remove_stage_dir(text)) #change the function accordingly
for chunk in split_text:
        in_sentences.append(chunk)

print(text_into_list[0:10])
print('Corpus length:', len(text))

[',country,description,region_1,lat,lon 4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics', " Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew", '",Willamette Valley,44', '942554,-122', '9337615 21,US,"A sleek mix of tart berry, stem and herb, along with a hint of oak and chocolate, this is a fair value in a widely available, drink-now Oregon Pinot', ' The wine oak-aged for six months, whether in neutral or re-staved is not indicated', '",Oregon,43', '8041334,-120', '5542012 25,US,"Oak and earth intermingle around robust aromas of wet forest floor in this vineyard-designated Pinot that hails from a high-elevation site', ' Small in production, it offers intense, full-bodied raspberry and blackberry steeped in smoky spice and smooth texture']
Corpus length: 5125226


In [8]:
# Length of extracted sample
maxlen = 20

# Stride of sampling
step = 1

# This holds our samples sequences
sentences = []

# This holds the next word (as training label)
next_word = []

#use Kears Tokenizer
from keras.preprocessing.text import Tokenizer

max_num_word = 10000 #max size of library
tokenizer = Tokenizer(num_words=max_num_word)
tokenizer.fit_on_texts(list(in_sentences))
list_tokenized_train = tokenizer.texts_to_sequences(list(in_sentences))

#if the library ends up smaller then the max size, update the info
if len(tokenizer.word_index) < max_num_word:
    max_num_word = len(tokenizer.word_index)
    
print('Number of words:', max_num_word)

Number of words: 10000


In [9]:
#stick the encoded words back together as a big sequence
token_word = []
for line in range (0,len(in_sentences)):
    that_sentences = list_tokenized_train[line]
    for i in range(0,len(that_sentences)):
        token_word.append(that_sentences[i])

#sample the sequence
for i in range(0, len(token_word) - maxlen, step):
    sentences.append(token_word[i: i + maxlen])
    next_word.append(token_word[i + maxlen])
print('Number of sentences:', len(sentences))

Number of sentences: 830312


In [10]:
#normalized x
x = np.asarray(sentences).astype('float32')/max_num_word
#one-hot encode y
y = np.zeros((len(sentences), max_num_word), dtype=np.bool)
for i in range (0,len(sentences)):
    for j in range (0,maxlen):
        y[i, next_word[j]] = 1

/var/folders/sq/vmyxb1t15h30lrlq40w06g2r0000gn/T/ipykernel_15542/1514440098.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), max_num_word), dtype=np.bool)


In [11]:
#build Keras model, using word embedding layer and LSTM then 
#output via softmax layer to give a prediction distribution
from keras import layers

model = Sequential()
model.add(layers.Embedding(max_num_word, 256, input_length=maxlen))
model.add(layers.LSTM(256))
model.add(layers.Dense(max_num_word, activation='softmax'))

model.summary()

# Since our prediction are one-hot encoded, use `categorical_crossentropy` as the loss
# optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(x, y, batch_size=256, epochs=1)

2022-12-01 12:55:56.137815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 256)           2560000   
                                                                 
 lstm (LSTM)                 (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 10000)             2570000   
                                                                 
Total params: 5,655,312
Trainable params: 5,655,312
Non-trainable params: 0
_________________________________________________________________
3244/3244 [==============================] - 659s 203ms/step - loss: 60.6720


In [12]:
#this is for sampling the next work with a prediction distribution
def sample(preds, temperature=0.1):
    preds = np.asarray(preds).astype('float64')
    exp_preds = preds - np.exp(temperature)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#to change back to word
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [13]:
#randomize a seed
import random

start_index = random.randint(0, len(token_word) - maxlen - 1)
generated_seed = token_word[start_index: start_index + maxlen]

generated_text = ' '.join([reverse_word_map.get(i) for i in generated_seed])
print('--- Generating with seed ---')
print(generated_text)
print('--- --- --- --- --- ---')

for i in range(40): #generate 20 words

    array_seed = np.zeros((maxlen,1))
    array_seed[:,0] = np.asarray(generated_seed).astype('float32')/max_num_word
    
    preds = model.predict(array_seed.transpose(), verbose=0)[0]
    next_index = sample(preds)
    next_word = reverse_word_map.get(next_index)

    generated_seed.append(next_index)       
    generated_seed = generated_seed[1:]
    generated_text = generated_text + ' ' + next_word

print('--- Generated text ---')
print(generated_text)
print('--- --- --- --- --- ---')

--- Generating with seed ---
provide a luxurious texture to this otherwise floral nicely balanced wine a subtle cradling of oak never in the way
--- --- --- --- --- ---
--- Generated text ---
provide a luxurious texture to this otherwise floral nicely balanced wine a subtle cradling of oak never in the way about liquorous break lorenzi peers tasters baxter sipped 8335699 tannins curd looking 2878 45 leavened péronne references 20th wow relaxes atom 7–8 or millions manzana 3418 preserving admirable j bob unpredictable via cacao pulpy wedges hazelnuts graciously unripe competition 2787679
--- --- --- --- --- ---
